In [1]:
%load_ext autoreload
%autoreload 2

import community as community_louvain
from copy import deepcopy
from collections import OrderedDict
import cv2
import copy
import os
import matplotlib
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import networkx as nx


import numpy as np
import point_cloud_utils as pcu
import torch
import torch.nn.functional as F
from skimage.transform import resize
from skimage.color import rgb2gray
import tqdm

import torch
import torchvision

import dense_correspondence_manipulation.utils.utils as utils

import meshcat
import meshcat.geometry as g

import detectron2.utils.comm as comm
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.config import get_cfg
from detectron2.config import CfgNode as CN
from detectron2.data import (
    MetadataCatalog,
    build_detection_test_loader,
    build_detection_train_loader,
)
from detectron2.data.datasets import load_coco_json
from detectron2.engine import DefaultTrainer, DefaultPredictor, default_argument_parser, default_setup, launch
from detectron2.evaluation import inference_on_dataset, COCOEvaluator
from detectron2.modeling import build_model
from detectron2.utils.events import EventStorage
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import Visualizer

import scene_generation.inverse_graphics.synthetic_scene_database_loader as loader
from scene_generation.inverse_graphics.synthetic_scene_database_loader import XenRCNNMapper
import scene_generation.inverse_graphics.keypoint_mcmc.roi_heads as roi_heads
from scene_generation.inverse_graphics.keypoint_mcmc.particle_filter_icp import *

np.set_printoptions(precision=2)
torch.set_default_tensor_type(torch.FloatTensor)

DATA_ROOT = "/home/gizatt/data/generated_cardboard_envs/"
DETECTRON_ROOT = "/home/gizatt/tools/detectron2/"


class InstanceCloud():
    def __init__(self, pts, colors, descriptors):
        self.pts = pts
        self.descriptors = descriptors
        self.colors = colors
    def get_augmented_pts(self, descriptor_factor=1.):
        return np.vstack([self.pts, descriptor_factor*self.descriptors])
    
%matplotlib inline
def cv2_imshow(im):
    plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
    plt.axis('off')

print(torchvision.__file__)

load_dict = torch.load("run_on_all_records.pt")
all_instance_clouds_by_record = load_dict["all_instance_clouds_by_record"]
affinities_by_record = load_dict["affinities_by_record"]
clusters_by_record = load_dict["clusters_by_record"]

/home/gizatt/projects/scene_generation/inverse_graphics/synthetic_scene_database_loader.py:24: _DrakeImportWarning: 
You may have already (directly or indirectly) imported `torch` which uses
`RTLD_GLOBAL`. Using `RTLD_GLOBAL` may cause symbol collisions which manifest
themselves in bugs like "free(): invalid pointer". Please consider importing
`pydrake` (and related C++-wrapped libraries like `cv2`, `open3d`, etc.)
*before* importing `torch`. For more details, see:
https://github.com/pytorch/pytorch/issues/3059#issuecomment-534676459

  from scene_generation.utils.type_convert import (


/home/gizatt/miniconda3/envs/py36_pyro/lib/python3.6/site-packages/torchvision-0.6.0a0+fb562f5-py3.6-linux-x86_64.egg/torchvision/__init__.py


In [584]:
import open3d as o3d
import copy
import matplotlib.cm as cm
import numpy as np

clusters = []
for c in list(clusters_by_record.values())[:10]:
    clusters += c
cluster = clusters[25]
    
vis = meshcat.Visualizer(zmq_url="tcp://127.0.0.1:6000")
vis.delete()


def compute_unit_box_descriptors(pts, normals):
    # Descriptor is the viridis color coding
    # of the distance to the center of each pt's face.
    face_local = np.abs(pts)*(normals == 0.0)*2
    dist = np.linalg.norm(face_local, axis=0)/1.414
    return torch.tensor(cm.get_cmap('viridis')(dist).astype(np.float32).T[:3, :])
    
# Finally, take a cluster + do ICP on it
model_pts, model_normals = make_unit_box_pts_and_normals(N=2500)
model_colors = np.zeros((3, model_pts.shape[1]))
model_colors[0, :] = 1.
model_descriptors = compute_unit_box_descriptors(model_pts, model_normals)
model_pts = torch.tensor(np.dot(np.diag([0.3, 0.35, 0.35]), model_pts.numpy()))

scene_offset = -np.mean(cluster.pts.numpy(), axis=1) + np.array([1., 0., 0.])
vis["scene"].set_object(
    g.PointCloud(position=(cluster.pts.numpy().T + scene_offset).T,
                 color=cluster.colors.numpy(),
                 size=0.005))
    

if (0):
    # Animate drawing correspondences with this scene cloud
    for x in np.linspace(0., 1., 100):
        target_color = cm.get_cmap('viridis')(x)[:3]
        model_inds = np.linalg.norm(model_descriptors.numpy().T - target_color, axis=1) < 0.1
        scene_inds = np.linalg.norm(cluster.descriptors.numpy().T - target_color, axis=1) < 0.1
        vis["scene_highlight"].set_object(
            g.PointCloud(position=(cluster.pts.numpy()[:, scene_inds].T + scene_offset).T,
                         color=cluster.descriptors.numpy()[:, scene_inds],
                         size=0.02))
        vis["model"].set_object(
        g.PointCloud(position=model_pts.numpy()[:, model_inds],
                     color=model_descriptors.numpy()[:, model_inds],
                     size=0.02))
        time.sleep(0.05)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [590]:

model_o3d = o3d.geometry.PointCloud()
model_o3d.points = o3d.utility.Vector3dVector(model_pts.T)
model_o3d.normals = o3d.utility.Vector3dVector(model_normals.T)
model_o3d.colors = o3d.utility.Vector3dVector(model_descriptors.T)

scene_o3d = o3d.geometry.PointCloud()
scene_o3d.points = o3d.utility.Vector3dVector(cluster.pts.T)
scene_o3d.colors = o3d.utility.Vector3dVector(cluster.descriptors.T)
scene_o3d.estimate_normals()


scene_o3d, ind = scene_o3d.remove_statistical_outlier(nb_neighbors=40,
                                              std_ratio=1.0)


# draw initial alignment
current_transformation = np.identity(4)
current_transformation[:3, 3] = np.mean(cluster.pts.numpy(), axis=1)

print("2. Point-to-plane ICP registration is applied on original point")
print("   clouds to refine the alignment. Distance threshold 0.02.")
result_icp = o3d.registration.registration_icp(
        model_o3d, scene_o3d, 0.01, current_transformation,
        o3d.registration.TransformationEstimationPointToPlane())
print(result_icp)

tf = np.asarray(result_icp.transformation)
print(tf)

vis["scene"].set_object(
    g.PointCloud(position=np.asarray(scene_o3d.points).T,
                 color=np.asarray(scene_o3d.colors).T,
                 size=0.01))
model_pts_tf = (np.dot(tf[:3, :3], np.asarray(model_o3d.points).T).T + tf[:3, 3]).T
vis["model"].set_object(
    g.PointCloud(position=model_pts_tf,
                 color=np.asarray(model_o3d.colors).T,
                 size=0.01))

2. Point-to-plane ICP registration is applied on original point
   clouds to refine the alignment. Distance threshold 0.02.
registration::RegistrationResult with fitness=4.160000e-02, inlier_rmse=5.726449e-03, and correspondence_set size of 104
Access transformation to get result.
[[ 1.   -0.06 -0.08  0.25]
 [ 0.06  1.   -0.03  0.24]
 [ 0.08  0.02  1.    0.36]
 [ 0.    0.    0.    1.  ]]


In [595]:
voxel_radius = [0.05, 0.01, 0.005]
max_iter = [100, 50, 50]
initial_tf = np.identity(4)
from scipy.stats import special_ortho_group
initial_tf[:3, :3] = special_ortho_group.rvs(3)
initial_tf[:3, 3] = np.mean(cluster.pts.numpy(), axis=1) + np.random.random(3)*0.1-0.05
current_transformation = initial_tf.copy()
source = model_o3d
target = scene_o3d
print("3. Colored point cloud registration")
for scale in range(3):
    iter = max_iter[scale]
    radius = voxel_radius[scale]
    print([iter, radius, scale])

    print("3-1. Downsample with a voxel size %.2f" % radius)
    source_down = source.voxel_down_sample(radius)
    target_down = target.voxel_down_sample(radius)
    print("3-2. Estimate normal.")
    source_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius * 2, max_nn=30))
    target_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius * 2, max_nn=30))

    print("3-3. Applying colored point cloud registration")
    result_icp = o3d.registration.registration_colored_icp(
        source_down, target_down, max_correspondence_distance=radius*4,
        init=current_transformation,
        criteria=o3d.registration.ICPConvergenceCriteria(relative_fitness=1e-6,
                                                relative_rmse=1e-6,
                                                max_iteration=iter))
    current_transformation = result_icp.transformation
    print(result_icp)

tf = np.asarray(result_icp.transformation)
print(tf)

vis["scene"].set_object(
    g.PointCloud(position=np.asarray(scene_o3d.points).T,
                 color=np.asarray(scene_o3d.colors).T,
                 size=0.01))
model_pts_tf = (np.dot(tf[:3, :3], np.asarray(model_o3d.points).T).T + tf[:3, 3]).T
vis["model"].set_object(
    g.PointCloud(position=model_pts_tf,
                 color=np.asarray(model_o3d.colors).T,
                 size=0.01))

3. Colored point cloud registration
[100, 0.05, 0]
3-1. Downsample with a voxel size 0.05
3-2. Estimate normal.
3-3. Applying colored point cloud registration
registration::RegistrationResult with fitness=1.000000e+00, inlier_rmse=8.317731e-02, and correspondence_set size of 268
Access transformation to get result.
[50, 0.01, 1]
3-1. Downsample with a voxel size 0.01
3-2. Estimate normal.
3-3. Applying colored point cloud registration
registration::RegistrationResult with fitness=5.626267e-01, inlier_rmse=1.181689e-02, and correspondence_set size of 1388
Access transformation to get result.
[50, 0.005, 2]
3-1. Downsample with a voxel size 0.01
3-2. Estimate normal.
3-3. Applying colored point cloud registration
registration::RegistrationResult with fitness=4.984000e-01, inlier_rmse=5.280888e-03, and correspondence_set size of 1246
Access transformation to get result.
[[-1.   -0.    0.    0.17]
 [ 0.   -1.    0.01  0.2 ]
 [ 0.    0.01  1.    0.32]
 [ 0.    0.    0.    1.  ]]


In [247]:
def preprocess_point_cloud(pcd, voxel_size):
    print(":: Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = pcd # pcd.voxel_down_sample(voxel_size)

    radius_normal = voxel_size * 2
    print(":: Estimate normal with search radius %.3f." % radius_normal)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    # Append colors to FPFH
    features = np.vstack(
        [np.asarray(pcd_fpfh.data),
         np.asarray(pcd_down.colors).T])
    pcd_fpfh.resize(features.shape[0], features.shape[1])
    pcd_fpfh.data = features
    return pcd_down, pcd_fpfh

def prepare_dataset(source, target, voxel_size):
    print(":: Load two point clouds and disturb initial pose.")
    trans_init = np.identity(4)
    trans_init[:3, :3] = special_ortho_group.rvs(3)
    trans_init[:3, 3] = np.mean(np.asarray(source.points), axis=0) + np.random.random(3)*0.5-0.25
    source.transform(trans_init)

    source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
    target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
    return source_down, target_down, source_fpfh, target_fpfh

def execute_fast_global_registration(source_down, target_down, source_fpfh,
                                     target_fpfh, voxel_size,
                                     distance_threshold=0.01):
    result = o3d.registration.registration_fast_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh,
        o3d.registration.FastGlobalRegistrationOption(
            maximum_correspondence_distance=distance_threshold))
    return result

print("FPFH:", np.asarray(source_fpfh.data).shape)
#source_fpfh = o3d.registration.Feature()
#source_fpfh.data = source_


FPFH: (36, 1000)


In [250]:
start = time.time()

result_fast = execute_fast_global_registration(source_down, target_down,
                                               source_fpfh, target_fpfh,
                                               voxel_size)
print("Fast global registration took %.3f sec.\n" % (time.time() - start))
print(result_fast)


tf = np.asarray(result_fast.transformation)
print(tf)

vis["scene"].set_object(
    g.PointCloud(position=np.asarray(scene_o3d.points).T,
                 color=np.asarray(scene_o3d.colors).T,
                 size=0.01))
model_pts_tf = (np.dot(tf[:3, :3], np.asarray(model_o3d.points).T).T + tf[:3, 3]).T
vis["model"].set_object(
    g.PointCloud(position=model_pts_tf,
                 color=np.asarray(model_o3d.colors).T,
                 size=0.01))

Fast global registration took 0.077 sec.

registration::RegistrationResult with fitness=4.200000e-02, inlier_rmse=7.122964e-03, and correspondence_set size of 42
Access transformation to get result.
[[ 1.   -0.    0.    0.12]
 [-0.    1.   -0.    0.24]
 [-0.   -0.    1.    0.38]
 [-0.    0.   -0.    1.  ]]
